# Qwen2.5-7B-Instruct Lora 微调
## 环境配置

在完成基本环境配置和本地模型部署的情况下，你还需要安装一些第三方库，可以使用以下命令：

```bash
python -m pip install --upgrade pip
# 更换 pypi 源加速库的安装
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple

pip install modelscope==1.18.0
pip install transformers==4.44.2
pip install streamlit==1.24.0
pip install sentencepiece==0.2.0
pip install accelerate==0.34.2
pip install datasets==2.20.0
pip install peft==0.11.1

```

## 模型下载
使用 modelscope 中的 snapshot_download 函数下载模型，第一个参数为模型名称，参数 cache_dir 为模型的下载路径。
Qwen2.5-7B模型大小为 15GB，下载模型大概需要 5 分钟。

In [1]:
import torch
from modelscope import snapshot_download, AutoTokenizer, AutoModelForCausalLM
import os
model_dir = snapshot_download('qwen/Qwen2.5-7B-Instruct', cache_dir='/root/autodl-tmp', revision='master')

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-02 15:02:24,703 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


## 数据格式化
`Lora`训练的数据是需要经过格式化、编码之后再输入给模型进行训练的，如果是熟悉`Pytorch`模型训练流程的同学会知道，我们一般需要将输入文本编码为`input_ids`，将输出文本编码为`labels`，编码之后的结果都是多维的向量。我们首先定义一个预处理函数，这个函数用于对每一个样本，编码其输入、输出文本并返回一个编码后的字典：

In [2]:

# tokenizer的位置是模型的根目录
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/qwen/Qwen2___5-7B-Instruct', use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/qwen/Qwen2___5-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model.enable_input_require_grads()

model.dtype

def process_func(example):
    MAX_LENGTH = 384   # 分词的过程中一个中文回切分成多个token
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


## 处理数据集

In [3]:
import pandas as pd
from datasets import Dataset
df = pd.read_json('/root/llm_practice/dataset/huanhuan.json')
ds = Dataset.from_pandas(df)

ds[:3]

tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3729/3729 [00:05<00:00, 684.17 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [4]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。<|endoftext|>'

In [5]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'嘘——都说许愿说破是不灵的。<|endoftext|>'

Tips： Qwen2中采用的`Prompt Template`格式如下：
```text
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你是谁？<|im_end|>
<|im_start|>assistant
我是一个有用的助手。<|im_end|>
```

## 定义LoraConfig
- `task_type`: 模型类别
- `target_modules`: 需要训练的模型层名字, 主要是`attention`部分的层, 不同的模型对应的层名字不同, 可以传入数组也可以传入字符串
- `r`: `lora`的秩
- `lora_alpha`: 微调后参数的权重比例

缩放的是`lora_alpha/r`

In [18]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'o_proj', 'gate_proj', 'down_proj', 'v_proj', 'q_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [19]:
model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


## 自定义TrainingArguments参数
- `output_dir`: 模型输出的路径
- `per_device_train_batch_size`: batch_size
- `gradient_accumulation_steps`: 梯度累加
- `logging_steps`: 多少步一个日志输出
- `num_train_epochs`: 多少个epoch
- `gradient_checkpointing`: 梯度检查, 一旦使用就需要执行`model.enable_input_require_grads()`

In [20]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
args = TrainingArguments(
    output_dir="./output/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)


In [21]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,3.757500
20,3.098600
30,3.061500
40,2.919100
50,2.900200
60,2.953600
70,2.912700
80,3.015100
90,3.046300
100,2.934000


/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter s

TrainOutput(global_step=699, training_loss=2.5736250897845485, metrics={'train_runtime': 1230.8599, 'train_samples_per_second': 9.089, 'train_steps_per_second': 0.568, 'total_flos': 4.575963998146867e+16, 'train_loss': 2.5736250897845485, 'epoch': 2.996784565916399})

## 合并模型

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = '/root/autodl-tmp/qwen/Qwen2___5-7B-Instruct'
lora_path = './output/Qwen2.5_instruct_lora/checkpoint-500'

# 加载tokenzier
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


温太医确实有些不同寻常，他虽是皇家御用的太医，却时常私下里来我们府上为娘娘们诊治。这温太医不仅医术高明，而且性情温和，待人谦逊，颇得大家喜爱。不过，也正因为如此，他的行为难免会引起一些人的猜疑。

我心中暗自思量：温太医为何如此频繁地光顾我们府上？难道他与我们之间有什么隐秘的关系？还是说，他另有目的？这些疑问在我心中盘旋，一时难以解开。

不过，温太医每次来时，总是带着一丝神秘的笑容，仿佛知晓了什么秘密一般。他为我诊治时，手法轻柔，言语间又总能给予我莫大的安慰和鼓励。或许，他只是真心想帮助我们，而不愿受制于那些繁文缛节的规矩。

然而，这样的想法终究只是猜测。我决定不再多想，专心调养身体，同时观察温太医的行为，看能否从中发现些什么端倪。毕竟，了解真相才能更好地保护自己和身边的人。
